In [ ]:
proj_dir='/path/to/main_project_folder/' # edit this line

nlag = 1
SNOWFREE = True # remove DJF from training data

import xarray as xr
import pandas as pd
import numpy as np
from tensorflow.random import set_seed as tf_set_seed
import tensorflow.compat.v1 as tf_compat_v1
import random
np.random.seed(101)
random.seed(201)
tf_set_seed(333)
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append(proj_dir)
from project_utils import parameters as param
from project_utils import load_region
from project_utils import prepare_inputs
from project_utils import utils as util
from project_utils import model_utils as mu
import importlib
importlib.reload(param)
importlib.reload(prepare_inputs)
importlib.reload(util)
importlib.reload(mu)
importlib.reload(load_region)
session_conf = tf_compat_v1.ConfigProto(device_count={'CPU': 24})
sess = tf_compat_v1.Session(config=session_conf)
import tensorflow.keras as keras
from matplotlib import colors

## calculate SM-T partial dependence plots for individual days (for use in Figure 3)

In [ ]:
region_list = ['southcentral_north_america']

ninputs = 2 # number of geospatial CNN input channels (i.e., GPH and SM)
w = 200 # window size for smoothing PDP curve
nlag = 1
SNOWFREE = True # remove DJF from training data

for dset in ["ERA5"]:
    for jj in range(len(region_list)):
        region_str = region_list[jj]
        hem, region_input_lat_bbox, region_input_lon_bbox, region_box_x, region_box_y, region_lat, region_lon, region_lon_EW, region_t62_lats, region_t62_lons = load_region.load_region_constants(region_str)        
        print(jj)
        print(region_str)
        print(hem)
        print(w)

        ### Read soilw data and sort
        pred_temps = pd.read_csv("../processed_data_"+dset+"/"+region_str+"/model_predictions_lag"+str(nlag)+".csv") 

        hgt_df = pd.read_csv("../processed_data_"+dset+"/"+region_str+"/region_avg_hgt.csv")
        hgt_df['time'] = pd.to_datetime(hgt_df['time'])
        hgt_df_test = pd.read_csv("../processed_data_"+dset+"/"+region_str+"/region_avg_hgt.csv")
        hgt_df_test['time'] = pd.to_datetime(hgt_df_test['time'])

        soilw_df = pd.read_csv("../processed_data_"+dset+"/"+region_str+"/region_avg_soilw_cday_anomaly.csv")
        soilw_df['time'] = pd.to_datetime(soilw_df['time'])
        time_vec = soilw_df['time']

        test_idx = pred_temps[pred_temps.set == 'test'].index.values  
        print(pred_temps)
        print(test_idx)
        if SNOWFREE:
            if hem == 'south':
                print('removing JJA soilw maps')
                soilw_df = soilw_df[(pd.to_datetime(time_vec).dt.month < 6) | (pd.to_datetime(time_vec).dt.month > 8)].iloc[:-1].reset_index(drop=True)
                hgt_df = hgt_df[(pd.to_datetime(time_vec).dt.month < 6) | (pd.to_datetime(time_vec).dt.month > 8)].iloc[1:].reset_index(drop=True)
                hgt_df_test = hgt_df_test[(hgt_df_test['time'].dt.month < 6) | (hgt_df_test['time'].dt.month > 8)].iloc[1:].reset_index(drop=True).loc[test_idx]
            elif hem == 'north':
                print('removing DJF soilw maps')
                soilw_df = soilw_df[(pd.to_datetime(time_vec).dt.month >= 3) & (pd.to_datetime(time_vec).dt.month <= 11)].reset_index(drop=True)
                hgt_df = hgt_df[(pd.to_datetime(time_vec).dt.month >= 3) & (pd.to_datetime(time_vec).dt.month <= 11)].reset_index(drop=True)
                hgt_df_test = hgt_df_test[(hgt_df_test['time'].dt.month >= 3) & (hgt_df_test['time'].dt.month <= 11)].reset_index(drop=True).loc[test_idx]
            print('hgt_df:', hgt_df)
            print('hgt_df_test:', hgt_df_test)
            print('soilw_df:', soilw_df)

        sorted_hgt_df = hgt_df.sort_values(by='hgt_anom_no_trend')
        sorted_hgt_df_test = hgt_df_test.sort_values(by='hgt_anom_no_trend')
        sorted_soilw_df = soilw_df.sort_values(by='soilw_daily_anom')
        print(sorted_hgt_df)
        print(sorted_hgt_df_test)
        print(sorted_soilw_df)

        ### Sort soilw input maps in order of increasing avg soilw anomaly
        input_dat, y_dat, ind, caldays, _time_vec = prepare_inputs.get_model_inputs(region_str, nlag, SNOWFREE, hemisphere=hem, dset=dset)
        hgt_input_dat = input_dat[:, :, :, 0]
        sorted_soilw_input_dat = input_dat[:,:,:,1][sorted_soilw_df.index.values, :, :]

        ## build model and load weights ##
        np.random.seed(101)
        random.seed(201)
        tf.random.set_seed(333)
        ninputs = 2
        ndense = 1
        ncnn = 1
        ncrp = 2
        conv_filters = 8
        dense_neurons = 32
        curr_reg = 0.001
        curr_loss = 'mean_squared_error'
        SNOWFREE = True
        curr_act_func = 'sigmoid'
        curr_opt = 'RMSprop'
        
        model = mu.build_model(lr = 1.0, conv_filters=conv_filters,
                               dense_neurons=dense_neurons, dense_layers = ndense, 
                               cnn_layers = ncnn, conv_relu_pool_layers = ncrp, 
                               activity_reg = curr_reg, input_channels = ninputs, 
                               loss_str=curr_loss, opt=curr_opt, 
                               act_func=curr_act_func, 
                               nlats=len(input_dat[0,:,0,0]), nlons=len(input_dat[0,0,:,0]))
        
        model.load_weights("../processed_data_"+dset+"/"+region_str+"/trained_weights"+"_lag"+str(nlag)+".h5")
        print(model.predict({"stacked_input" : input_dat, "calday": caldays}))

        #############################################
        # determine hgt days: high/med/low pressure #
        #############################################
        min_hgt_idx = sorted_hgt_df_test.index.values[0]
        med_hgt_idx = sorted_hgt_df_test.index.values[int(np.floor(len(sorted_hgt_df_test)/2))]
        max_hgt_idx = sorted_hgt_df_test.index.values[-1]
        print(min_hgt_idx, med_hgt_idx, max_hgt_idx)
        print(hgt_df.loc[min_hgt_idx])
        print(hgt_df.loc[med_hgt_idx])
        print(hgt_df.loc[max_hgt_idx])

        ############################################
        # determine hgt days: best hit / best miss #
        ############################################
        print(pred_temps)
        best_hit = (pred_temps.predicted_tmax - pred_temps.true_y).loc[test_idx].abs().idxmin()
        worst_miss = (pred_temps.predicted_tmax - pred_temps.true_y).loc[test_idx].abs().idxmax()
        print((pred_temps.predicted_tmax - pred_temps.true_y).loc[test_idx][best_hit])
        print((pred_temps.predicted_tmax - pred_temps.true_y).loc[test_idx][worst_miss])
        hgt_day_idx = [min_hgt_idx, med_hgt_idx, max_hgt_idx, best_hit, worst_miss] 
        idx_titles = ['low pressure', 'median pressure', 'high pressure', 'best hit', 'worst miss'] 

        ########################################################
        # ICE plots (Individual Conditional Expectation Plots) #
        ########################################################
        predict_df_daily = []
        x_vec = sorted_soilw_df.soilw_daily_anom.values
        ice_dict = {}
        fig, axes = plt.subplots(len(hgt_day_idx),1,figsize=(8,20))
        ax = axes.flatten()

        for ii,hgt_day in enumerate(hgt_day_idx):
            hgt_layer = hgt_input_dat[hgt_day, :, :].reshape(1, len(input_dat[0,:,0,0]), len(input_dat[0,0,:,0]))
            hgt_input = np.tile(hgt_layer, [len(sorted_soilw_input_dat), 1, 1])
            cday_input = np.tile(caldays[hgt_day], [len(sorted_soilw_input_dat), 1, 1])
            pdp_input = np.stack((hgt_input, sorted_soilw_input_dat[:,:,:]), axis=3)

            ########  model.predict #############
            tmax_predictions = model.predict({"stacked_input" : pdp_input, "calday": cday_input})[:,0]
            cumsum_vec = np.cumsum(np.insert(tmax_predictions, 0, 0)) 
            ma_vec = (cumsum_vec[w:] - cumsum_vec[:-w]) / w

            ########  make the pdp plot ##########
            ax[ii].scatter(x_vec, tmax_predictions, 
                       color = (68/255, 2/255, 86/255), s = 1.5, zorder = 0)
            p = ax[ii].hist2d(x_vec, tmax_predictions, bins = 35, cmin = 10, 
                      norm = colors.LogNorm(), zorder = 5, cmap = "twilight_shifted");
            ax[ii].scatter(x_vec[int(0.5*w):int(-0.5*w)],ma_vec[:-1], s=1, c='r', zorder=10)
            ax[ii].scatter(soilw_df.soilw_daily_anom.iloc[hgt_day], pred_temps.true_y.iloc[hgt_day], s=60, c='lime', marker='*', zorder=10)
            ax[ii].scatter(soilw_df.soilw_daily_anom.iloc[hgt_day], pred_temps.predicted_tmax.iloc[hgt_day], s=60, c='pink', marker='*', zorder=10)
            ax[ii].set_title(idx_titles[ii]+'\n'+str(sorted_soilw_df.time[hgt_day]))
            ax[ii].set_xlabel('region avg soilw anomaly')
            ax[ii].set_ylabel('Predicted TMAX (K)')

            ice_dict[idx_titles[ii]] = {'hgt_day_idx': hgt_day.astype(float),
                                                'hgt_layer': hgt_layer[0,:,:].tolist(),
                                                'ice_x_vec' : x_vec[int(0.5*w):int(-0.5*w)].tolist(),
                                                'ice_y_vec' : ma_vec[:-1].tolist(),
                                                'soilw_vec' : x_vec.tolist(),
                                                'Predicted TMAX' : tmax_predictions.tolist(),
                                                'date' : str(sorted_soilw_df.time[hgt_day]),
                                                'avg_sm' : soilw_df.soilw_daily_anom.iloc[hgt_day],
                                                'reanalysis_tmax' :  pred_temps.true_y.iloc[hgt_day],
                                                'model_predicted_tmax' :  pred_temps.predicted_tmax.iloc[hgt_day]}
        plt.tight_layout()
        plt.show()

        import json
        with open("../processed_data_"+dset+"/"+region_str+"/ice_plot_data_fig3.txt", 'w') as f:
            f.write(json.dumps(ice_dict))